<a href="https://colab.research.google.com/github/Alan-Cheong/IEEE_QW_2020/blob/master/Simulated_Annealing_Office_Rents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = "your_file_path_here.csv"  # Update this with the actual path
df = pd.read_csv(file_path)

# Ensure the date column is in datetime format
df['Quarter'] = pd.to_datetime(df['Quarter'])

# Sort by date (just in case)
df = df.sort_values(by='Quarter')

# Extract rents and vacancy rates
rents = df['Rent'].values
vacancies = df['Vacancy'].values

from scipy.interpolate import interp1d

# Create an interpolation function based on historical data
vacancy_function = interp1d(rents, vacancies, kind='linear', fill_value='extrapolate')

# Define cost function (uses real data)
def vacancy_rate(rent_price):
    return vacancy_function(rent_price)  # Use actual vacancy data

def simulated_annealing(initial_rent, temp, cooling_rate, max_iterations):
    current_rent = initial_rent
    best_rent = current_rent
    best_cost = vacancy_rate(current_rent)

    for i in range(max_iterations):
        new_rent = current_rent + np.random.uniform(-5, 5)  # Small perturbation
        new_cost = vacancy_rate(new_rent)

        # Accept new rent price if it's better, or with probability if worse
        if new_cost < best_cost or np.exp((best_cost - new_cost) / temp) > np.random.rand():
            current_rent = new_rent
            best_cost = new_cost
            best_rent = new_rent

        # Cooling
        temp *= cooling_rate

        # Stop if temp is too low
        if temp < 1e-3:
            break

    return best_rent, best_cost

# Run optimization with real data
optimal_rent, optimal_vacancy = simulated_annealing(initial_rent=np.mean(rents), temp=1000, cooling_rate=0.95, max_iterations=1000)

print(f"Optimal Rent: ${optimal_rent:.2f} per sq. ft")
print(f"Vacancy Rate at Optimal Rent: {optimal_vacancy:.4f}")

